In [1]:
import gc
import sys
from statistics import mean
import time
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import itertools
from pathlib import Path
from scipy import special
import matplotlib.pyplot as plt
import datetime
import torch.linalg as LA
from torch.distributions.laplace import Laplace



import utils
import models
import learning_utils
from configurations import args_parser, arguments



In [3]:
%load_ext autoreload
%autoreload 2


start_time = time.time()
args = arguments()
#boardio is for the the tensorboardx prensation and textio is for written documentation
boardio, textio, best_val_acc, path_best_model, last_model_path = utils.initializations(args)
textio.cprint(str(args) if args.__class__.__name__ == 'Namespace' else str(vars(args)))

#mnist_train_data, mnist_test_loader  = utils.data(args)





{'exp_name': None, 'eval': False, 'data': 'mnist', 'norm_std': 0.5, 'norm_mean': 0.5, 'train_batch_size': 20, 'test_batch_size': 1000, 'model': 'cnn2', 'num_users': 30, 'num_users_per_round': 5, 'local_epochs': 1, 'local_iterations': 100, 'global_epochs': 200, 'tau_min': 0.05, 'privacy_noise': 'laplace', 'epsilon_bar': 400, 'optimizer': 'sgd', 'lr': 0.01, 'momentum': 0.5, 'lr_scheduler': False, 'device': device(type='cpu'), 'seed': 0, 'zeta_coeff': 1.05, 'alpha': 5, 'beta': 2, 'gamma': 0.1, 'max_seconds': 300, 'method_choosing_users': 'ALSA', 'data_truncation': 1750, 'choosing_users_verbose': False, 'save_best_model': False, 'privacy': True, 'privacy_choosing_users': True, 'epsilon_sum_deascent_coeff': 0.02, 'delta_f': 0.0008, 'snr_verbose': True, 'max_iterations_alsa': 100}


In [3]:
# data
train_data, test_loader = utils.data(args)
#input in the CNNs is the number of channels and in linear models is the size of the flatten pictures
input, output, train_data, val_loader = utils.data_split(train_data, len(test_loader.dataset), args)

# model
if args.model == 'mlp':
    global_model = models.FC2Layer(input, output)
elif args.model == 'cnn2':
    global_model = models.CNN2Layer(input, output, args.data)
elif args.model == 'cnn3':
    if args.data == 'cifar10':
        global_model = models.CNN3LayerCifar()
    else:
        global_model = models.CNN3LayerMnist()
elif args.model == 'cnn5':
    if args.data == 'mnist' or args.data == 'fashion mnist':
        raise ValueError('CNN5 is not supported for MNIST type datasets')
    global_model = models.CNN5Layer(input, output)
elif args.model == 'linear':
    global_model = models.Linear(input, output)



textio.cprint(str(summary(global_model)).encode('utf-8', errors='ignore').decode('utf-8', errors='ignore'))
global_model = global_model.to(args.device)
print(f"global model's device: {next(global_model.parameters()).device}")

train_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
test_criterion = torch.nn.CrossEntropyLoss(reduction='sum')

local_models = utils.federated_setup(global_model, train_data, args, i_i_d=True)
utils.update_data_equility_partititon(local_models, args)





Layer (type:depth-idx)                   Param #
CNN2Layer                                --
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            906
├─Linear: 1-3                            4,850
├─Linear: 1-4                            510
├─Dropout: 1-5                           --
├─Dropout: 1-6                           --
├─BatchNorm2d: 1-7                       12
├─BatchNorm1d: 1-8                       100
Total params: 6,534
Trainable params: 6,534
Non-trainable params: 0
global model's device: cpu


In [4]:
# for i in range(100):
#     local_models[0].update_privacy_violation_and_reward()
#     print(f"iteration {i}, next_privacy_term is {local_models[0].next_privacy_term}")

In [5]:


choices_table = np.zeros((args.global_epochs, args.num_users))
num_of_obs_arr = np.zeros((1,args.num_users))
train_loss_list = []
val_acc_list = []
val_losses_list = []
l1_norm_avg_deltha_theta_list = []
global_epochs_time_list = []


time_counter = 0
for global_epoch in tqdm(range(1, args.global_epochs+1)):
    """Part 1: Choosing Users"""
    for usr_idx in range(args.num_users):
        local_models[usr_idx].update_g(global_epoch)
        local_models[usr_idx].update_ucb(global_epoch)
    
    if args.choosing_users_verbose:
        print(f"iteration{global_epoch}")
    a=time.time()
    rounds_choise = utils.choose_users(local_models, args, global_epoch, method=args.method_choosing_users)

    
    #choices_table[global_epoch-1, rounds_choise] = 1
    num_of_obs_arr[0,rounds_choise] += 1
    for usr_idx in rounds_choise:
        local_models[usr_idx].update_emp_avg()
        local_models[usr_idx].update_privacy_violation_and_reward()
        local_models[usr_idx].increase_num_of_obs()
        if args.choosing_users_verbose:
            print(f"user {usr_idx}, g: {local_models[usr_idx].g}, ucb: {local_models[usr_idx].ucb}, num_of_obs: {local_models[usr_idx].num_of_obs}, privacy reward: {local_models[usr_idx].privacy_reward}, curr_delay = {local_models[usr_idx].last_access_time}")
    
    max_delay = max([local_models[i].last_access_time for i in rounds_choise])
    if args.choosing_users_verbose:
        print(f"max_delay = {max_delay:.2f} seconds")
    
    
    """Part 2: Training"""
#     learning_utils.distribute_model(local_models, global_model)
#     users_avg_loss_over_local_epochs = []

#     for user_idx in rounds_choise:
#         user_loss = []
#         for local_epoch in range(args.local_epochs):
#             user = local_models[user_idx]
#             train_loss = learning_utils.train_one_epoch(user, train_criterion, args)
#             if args.lr_scheduler:
#                 user.scheduler.step(train_loss)
#             user_loss.append(train_loss)
#         users_avg_loss_over_local_epochs.append(mean(user_loss))
    
#     avg_loss_over_chosen_users_curr_global_epoch = mean(users_avg_loss_over_local_epochs)
#     train_loss_list.append(avg_loss_over_chosen_users_curr_global_epoch)


#     avg_deltha_theta = learning_utils.Fed_avg_models(local_models, global_model, rounds_choise,
#                                                       args, snr_verbose = args.snr_verbose)
#     # l1_norm_avg_deltha_theta = sum(LA.vector_norm(param.flatten(),1) for param in avg_deltha_theta.values())
#     # l1_norm_avg_deltha_theta_list.append(l1_norm_avg_deltha_theta)
#     # print(f"l1_norm_avg_deltha_theta is {l1_norm_avg_deltha_theta}")
    

#     val_acc, val_loss = learning_utils.test(test_loader, global_model, test_criterion, args)
#     val_acc_list.append(val_acc) ; val_losses_list.append(val_loss)
    

#     # boardio.add_scalars("Losses over time in seconds", {"train_loss":avg_loss_over_chosen_users_curr_global_epoch,
#     #                                     "val_loss": val_loss}, time.time()-start_time)
#     # boardio.add_scalar('Val Accuracy', val_acc, time.time()-start_time)


#     time_counter += max_delay
#     print((f"global epoch {global_epoch} has been done artifficialy in {max_delay:.2f} secs, the total time by now is {time_counter:.2f} \n with avg train loss {avg_loss_over_chosen_users_curr_global_epoch:.3f}, val loss {val_loss:.3f}, avg val acc {val_acc:.2f}%"))
#     global_epochs_time_list.append(time_counter)
#     gc.collect()


#     if val_acc > best_val_acc and args.save_best_model:
#         best_val_acc = val_acc
#         torch.save({"model's state dict":global_model.state_dict(),
#                     "train_loss_list": train_loss_list,
#                     "val_acc_list": val_acc_list,
#                     "val_losses_list": val_losses_list,
#                     "global_epochs_time_list": global_epochs_time_list,
#                     "num_of_users": args.num_users,
#                     "num_of_users_per_round": args.num_users_per_round,
#                     "l1_norm_avg_deltha_theta_list": l1_norm_avg_deltha_theta_list}
#                     , path_best_model)
    
    
#     torch.save({"model's state dict":global_model.state_dict(),
#                 "train_loss_list": train_loss_list,
#                 "val_acc_list": val_acc_list,
#                 "val_losses_list": val_losses_list,
#                 "global_epochs_time_list": global_epochs_time_list,
#                 "num_of_obs_arr": num_of_obs_arr.reshape(-1),
#                 "global_epoch": global_epoch,
#                 "num_of_users": args.num_users,
#                 "num_of_users_per_round": args.num_users_per_round,
#                 "l1_norm_avg_deltha_theta_list": l1_norm_avg_deltha_theta_list}
#                 , last_model_path)

#     if time_counter > args.max_seconds:
#         break

# users_idxs = tuple([str(x) for x in range(1,args.num_users+1)])
# fig, ax = plt.subplots()
# ax.bar(users_idxs, num_of_obs_arr.reshape(-1), width = 0.4)
# ax.set_title("Number of times each user was chosen")
# ax.set_ylabel("number of times")
# ax.set_xlabel("user index")
# #boardio.add_figure("Number of times each user was chosen", fig, global_epoch)
# plt.savefig(last_model_path.parent / "Number of times each user was chosen.png")

# #choices_table = choices_table.cumsum(axis=0)


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [55:55<00:00, 16.78s/it]   
